In [1]:
import pandas as pd
import csv
import re
import concurrent.futures
import emoji
from collections import Counter

In [2]:
re.sub(r'(.)\1{2,}', r'[xd]', "haaaaapppppyyy")

'h[xd][xd][xd]'

In [3]:
re.sub(r'(.)\1{3,}', r'\1\1\1', "haaaappppyyyy11111111")

'haaapppyyy111'

In [4]:
re.sub(r'(.)\1{2,}', r'\1\1', "haaaaapppppyyy")

'haappyy'

In [5]:
re.sub(r'([A-Za-z])\1{3,}', r'\1\1\1', "haaaappppyyyy11111111@@@@@@#####🙄🙄🙄🙄🙄🙄 aaaaaaaaaaaaaaa")

'haaapppyyy11111111@@@@@@#####🙄🙄🙄🙄🙄🙄 aaa'

In [6]:
#import csv
filename = 'tweets5utf8'
extension = '.csv'
tweets_df = pd.read_csv(filename+extension, index_col=0)
tweets_df['cleaned'] = "" 
tweets_df.head()

,text,date,lang,author,cleaned
0,Hindi maling tao. Maling pagkakataon siguro ht...,2020-04-25T07:16:39.000Z,tl,1008799086038769665,
1,Resign or Loa??,2020-04-25T07:16:42.000Z,en,871745373823352832,
2,Happy Birthday!! @Carlboiiii1 🥳,2020-04-25T07:16:43.000Z,en,1123217470553149440,
3,@jiecueto Wait aabot ako diyan mamaya ep6 nko.,2020-04-25T07:16:46.000Z,tl,190977737,
4,Haha muka nga🤣🤣🤣 https://t.co/aVUsCigtj3,2020-04-25T07:16:47.000Z,tl,1026836291025915907,


In [7]:
# %%time
# mentions = r"@[^ ]+"
# links = r"(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+"

# for i in range(500):
#     mltext = tweets_df.iloc[i].text
#     ltext = re.sub(mentions, '[mention]', mltext)
#     ftext = re.sub(links, '[link]', ltext)
#     tweets_df["text"].replace({tweets_df.iloc[i].text : ftext}, inplace=True)
#     #print(i, tweets_df.iloc[i].text) # can comment this out
    
# # 36 seconds for 500 tweets

In [8]:
%%time
mentions = r"@[^ ]+"
links = r"(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+"
repeating = r'([A-Za-z])\1{3,}'
foreign = r"[^\x00-\xff\u2000-\u26FF]" #Non Ascii Character or Unicode Symbols

for i in range(len(tweets_df)):
    text = str(tweets_df.at[i, 'text'])
    text = text.lower() # lowercase
    text = re.sub(mentions, '<mention>', text) # convert mentions to <mentions>
    text = re.sub(links, '<link>', text) # convert links to <links>
    text = re.sub(repeating, r'\1\1\1', text) # normalize
    text = emoji.demojize(text) # convert :) to :happy:
    text = re.sub(foreign, '<foreign>', text) # convert all non ascii to <foreign>
    text = emoji.emojize(text) # convert back :happy: to :) emoji
    tweets_df.at[i, 'cleaned'] = text  # update the row text
    #print(i, tweets_df.at[i, 'text']) # can comment this out

# without demojize and emojize step
# ~170millisec for 500 tweets
# ~17s for full(500k+)

# with demojize and emojize step
# 3.5mins - 4mins for full(500k+)

Wall time: 3min 20s


In [9]:
tweets_df

,text,date,lang,author,cleaned
0,Hindi maling tao. Maling pagkakataon siguro ht...,2020-04-25T07:16:39.000Z,tl,1008799086038769665,hindi maling tao. maling pagkakataon siguro <l...
1,Resign or Loa??,2020-04-25T07:16:42.000Z,en,871745373823352832,resign or loa??
2,Happy Birthday!! @Carlboiiii1 🥳,2020-04-25T07:16:43.000Z,en,1123217470553149440,happy birthday!! <mention> 🥳
3,@jiecueto Wait aabot ako diyan mamaya ep6 nko.,2020-04-25T07:16:46.000Z,tl,190977737,<mention> wait aabot ako diyan mamaya ep6 nko.
4,Haha muka nga🤣🤣🤣 https://t.co/aVUsCigtj3,2020-04-25T07:16:47.000Z,tl,1026836291025915907,haha muka nga🤣🤣🤣 <link>
...,...,...,...,...,...
555911,Amen 😇 https://t.co/8pbz4te5sR,2020-05-05T05:21:46.000Z,en,26676581,amen 😇 <link>
555912,@xmoodiexx Wag na lng dhen ako tatawa sa susunod,2020-05-05T05:21:48.000Z,tl,1246792944247398401,<mention> wag na lng dhen ako tatawa sa susunod
555913,ambata pa pala https://t.co/mre2L9tWg2,2020-05-05T05:21:49.000Z,tl,719335525740511232,ambata pa pala <link>
555914,After meeting; stress pressure pasok,2020-05-05T05:21:52.000Z,en,542330477,after meeting; stress pressure pasok


In [10]:
#save to csv
tweets_df[['text','cleaned', 'date', 'lang', 'author']].to_csv(filename+'_cleaned_and_raw'+extension, encoding='utf-8', quoting=csv.QUOTE_ALL)
tweets_df[['cleaned', 'date', 'lang', 'author']].to_csv(filename+'_cleaned'+extension, encoding='utf-8', quoting=csv.QUOTE_ALL)

In [11]:
# # To test kung ano yung tweets na may foreign
# # Some tweets dito in full english naman pero in different font

# #for i in range(len(tweets_df)):
# for i in range(100000):
#     text = tweets_df.at[i, 'text']
#     text = emoji.demojize(text) # convert :) to :happy:
#     found = re.search(foreign, text)
#     if found:
#         print(i, text)

84 @Anggeeeelll 𝚒𝚕𝚘𝚟𝚎𝚢𝚘𝚞𝚝𝚘𝚘:beating_heart::smiling_face_with_heart-eyes::face_blowing_a_kiss:
132 5 𝚢𝚎𝚊𝚛𝚜 𝚊𝚗𝚍 5 𝚖𝚘𝚗𝚝𝚑𝚜:beating_heart: https://t.co/VXuzhNz0HR
310 HAHAHA 감사합니다 :smiling_face_with_smiling_eyes: https://t.co/VOE5XYwnfX
639 Rewatching #คั่นกูEP10
@bbrightvc his expression shifting is too sexy :red_heart: https://t.co/IkusUPGoNl
819 ʀᴇᴀɴɪᴍᴀᴛɪᴏɴ
:people_wrestling: @TripleH
:office_building: @WWE

#TripleH #TheGame #IAmTheGame #TripleH25 #WWE #RAW #SmackDown #WrestlingArt #FanArt #DigitalArt #DigitalPainting #Chibi #Caricature #PhotoshopEdit

#MoiMagnum:registered: #Reanimations:trade_mark: :copyright:2020 https://t.co/PVr8mvmrp9
822 훌륭한 :two_hearts: https://t.co/LoEuzCt0IN
1183 CREATED NEW PRESET! NEW PRESET for  #คั่นกูEP10 ctto @winmetawin ig :orange_heart::red_heart: https://t.co/DmqLedF3mt
1216 Rica Peralejo has spoken. TopTap is shaking. @Toptap_jirakit 

The major comeback of a Filipina actress in a Thai BL series, 2gether The Series.

#คั่นกูEP10 #2gethertheseriesep10


8567 รู้สึกเหมือนจริงมาก :smiling_face_with_hearts::smiling_face_with_hearts::smiling_face_with_hearts: https://t.co/qmwMmujwl3
8665 แสดง แค่รัก ร้องไห้. มีความสัมพันธ์. #Ep10 #sarawattine #inarealationship @ Quezon City, Philippines https://t.co/kKVlgWNVQX
8697 :red_circle: NEW VLOG! NOOD NA MGA BALUGS!

LINK ✓
https://t.co/9MqvXbtIbM https://t.co/QANGljRgJA
8771 何度も何度も言います、#StayHome!
私たち医療関係者は不死ではありません。 
戻りたい場所があります。
一緒にいたい家族がいます。 
大切な命があります。 
私たちは見たい先物があります。 
私たちを家に帰れるようにお手伝ってください。
9380 𝙴𝚙𝚒𝚜𝚘𝚍𝚎 7: 𝚃𝚘𝚞𝚌𝚑 𝚢𝚘𝚞𝚛 𝚑𝚎𝚊𝚛𝚝 :heart_suit:
9457 الفلبين :red_heart::smiling_face_with_heart-eyes: @ Siquijor Island, Philippines https://t.co/VqBDCIrKha
9524 ρѕт ρωє∂є мσ вα кυиg ѕαмαнαи ∂ιтσ кαѕє ѕσвяαиg ѕαкιи иα тαℓαgα єιιι gιиαωα кσ иαмαи ℓαнαт ραяα ѕαкαиуα ρєяσ вαт gαиσи αкσ ραяιи уυиg мαℓιиg тασ нммммммм мαнαℓ кσ иαмαи ѕуα ρєяσσσ тαиσиg кσ иgα αкσ ραяιи вα уυиg мαнαℓ иуα ѕα ιѕιρ кσ иgαуσи ѕαиα αкσ ραяιи кαуα тнαикѕ ѕα тιмє
9594 Waiting for episode 11...ㅋㅋㅋㅋㅋ https://t.co/28QXLjbIQW
9646 [웨이보] BTS_

17295 sparring session  #คั่นกูEP10 https://t.co/tB0qeWnW6p
17454 #텔레비전에_재환이가_나와요
#텔레그나_아홉시_째니나온다 https://t.co/Rmol77Sr7s
17456 @Shinigaengg ㅋㅋㅋㅋㅋㅋㅋ
17545 #텔레비전에_재환이가_나와요
#텔레그나_아홉시_째니나온다 https://t.co/VhgqAeBT9f
17797 𝐵𝑒𝒶𝓊𝓉𝓎  𝒾𝓈  𝓉𝒽𝑒  𝒾𝓁𝓁𝓊𝓂𝒾𝓃𝒶𝓉𝒾𝑜𝓃  𝑜𝒻  𝓎𝑜𝓊𝓇  𝓈𝑜𝓊𝓁  :sunflower: @ Solano, Nueva Vizcaya https://t.co/qGz4n3zfkB
17862 Привяу, поболтаем?.. :rolling_on_the_floor_laughing: Напиши, что думаешь! Уже в следующую пятницу я проведу первую встречу разговорного клуба «English Please», и вот думаю насчёт темы. Нравится идея поговорить о том, зачем вы вообще учите… https://t.co/FvLP3WyW4F
18086 ไม่มีอะไร ... แค่ saifah และ zon ที่รักกัน (อาจจิมกับทอมอยู่แล้ว) ...... 
 #สายฟ้าซนยังไม่จบ #Mii2 #WHYRUtheseries #WHYRULastEP https://t.co/8Je6ppN7uE
18144 @essecn @xiyan68 如果在基督里，你这知道人从哪里来到哪里去了，他会指引你前进的方向。
18154 𝑳𝒐𝒗𝒆 𝒐𝒇 𝑴𝒂𝒚 𝑺𝒆𝒋𝒖𝒏 𝒊𝒏 𝑴𝒂𝒏𝒊𝒍𝒂 :strawberry:
𝐅𝐑𝐄𝐄 𝑨𝒊𝒓𝒉𝒐𝒍𝒅𝒆𝒓 𝑬𝒗𝒆𝒏𝒕 𝒇𝒐𝒓 𝑳𝒊𝒎 𝑺𝒆𝒋𝒖𝒏 ♡
𝑆𝑝𝑒𝑐𝑖𝑎𝑙 𝑡ℎ𝑎𝑛𝑘𝑠 𝑡𝑜, @myseungjun :Thailand:
♡ 𝑩𝒂𝒏𝒈𝒌𝒐𝒌 𝒙 𝑴𝒂𝒏𝒊𝒍𝒂 ♡
𝐒𝐥𝐨𝐭: https://t.co/wI6rPAKmXf
#세준 #임세준 #VICT

29677 오늘도 고생 많이 했어요 준면아! 우리 리더 항상 고마워요!! 너도 잘 자요. 오래오래 사랑하자!!! :smiling_face_with_hearts::red_heart:
29730 @H1aK1LAIwSthwOZ なんか、すごい事なりそう！リアルにしたら
30014 @Allbrightvc #คั่นกูEP10
30022 @Allbrightvc #คั่นกูEP10
30042 @eleynorbna Aļam mo yęrnnn
30623 굿토 코 시양 이-잩 (。_。)
30895 카ㅣ닛
31211 @WE_THE_BOYZ ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ :smiling_face_with_open_hands::smiling_face_with_halo: 찬희야 생일축하해.....요
31312 YAWAAA SARAWAT!!!  AGGRESIVE TOP MAN!!! Yummmmm!!!!  #คั่นกูEP10
31474 ہر شخص ہے خدا بننے میں مصروف 
یہ تماشہ بھی خدا دیکھ رہا ہے

Respect &amp; Support for Tariq Jamil Sahab.
@TariqJameelOFCL https://t.co/VbmAyqt21J
31538 #MyMemoriesWithPTay is taken way back 2017, they surprise (my P'Aluin and AJ) meeting P'Tay for the first time imissu. 

@taytawanfc
@everytayph
@ALLABOUTTAYNEWS
@taynationph

#Tawan_V 
#taytawan 
#taytawanfc 
#YourEveyTay 
#TawanIsLife 
#ชาวบ้าน 
#GMMTV https://t.co/6VcFPVbJdb
31961 @knm_pinas それ、まじで微妙ですよね:face_with_tears_of_joy:笑
32464 this scene is not for SarawatTine... I'm gonna tell yo

40584 Seseorang bertanya kepada Abu Darda' رضي الله عنه, "Ajarkanlah kepadaku satu nasehat yang dengan itu  Allah selipkan manfaat untukku...?." Beliau menjawab, "Urusan kehormatanmu serahkanlah kepada Allah, siapapun yang mempermalukanmu atau menghinamu biarlah Allah yg membalasnya." https://t.co/f6Gi9aouZl
40697 Loving the new TWICE comeback 

[JTBC 브랜드송] 컬러풀 JTBC ♪ (트와이스 ver.) https://t.co/HtP9ruT7ab via @YouTube
40820 ㅋㅋㅋ555 mao na imong katawa pag half korean half thai ka. Char ok next
40912 feliz cumpleaños @uniquesalongaph ㋡
41501 say you'll share with me one love, one lifetime￼
41551 Auto Focus Eyes Activate!
#คั่นกูEP10 #brightwin https://t.co/CIhNR89br5
41637 So I was bored and got nothing to do on Tiktok so I made this :face_with_tears_of_joy::face_with_tears_of_joy::face_with_tears_of_joy:
They will never let you go:face_with_tears_of_joy:

 @pledis_17 #세븐틴 #SEVENTEEN https://t.co/p3WLFUkef0
41642 Zzzzzz ✘:frowning_face:
41897 hoy！:pleading_face: https://t.co/IiFbyYJayw
420

50428 I know u guys r in love :smiling_face_with_hearts: #คั่นกู #เพราะเราคู่กัน #2getherTheSeries #bbrightvc #brightwin #ทีมเมียสารวัตร  #winmetawin  #snowballpower https://t.co/Sdo62hdHxm
50521 ما أعرف ليش الناس تتمنى الشر لغيرها و تكيد و تخطط عشان تأذي ، لكن كيد الله أعظم فأسأل الله أنه يكيد لي من كل شخص يحاول أذيتي و يرُد كيده في نحره.
50603 sharing my recent played songs on YT! Support local (θ‿θ) https://t.co/q3SCl3HmGj
50675 @xlainm oops ( ͡° ͜ʖ ͡°) https://t.co/8pigcaA76j
50982 @Alwaafy11Khal لبسها كمامه وغطي عيونها ولف فيها بالبيت
51091 . . . wedding pictures. They’re magical. Get a better look at the hand embroidered jusi Noel Barong Tagalog that Drew wears here at https://t.co/DSYBxVkIwe. Photo by @muscovadoproject (on Instagram)⠀⠀⠀⠀⠀⠀⠀⠀⠀ #barongtagalog #barong #barongs  #filipiniana #filipiñiana
51171 𝙶𝚘𝚘𝚍 𝚖𝚘𝚛𝚗𝚒𝚗𝚐!
51508 ฉันเก็บทุกอย่างเกี่ยวกับคุณ
51604 @zhaozhu01 是狗熊
51758 اعتقد اني اعيش لحالي 
بمدينة مجهوله بجزيرة باخر العالم
صعب جدا  
فكر معي
But we well make it
51951 @

58954 :megaphone::megaphone::megaphone:Want to see the cute p'tay?
Let's wait for tonight's broadcast:blue_heart:

:men_holding_hands:：@Tawan_V  @mynameisnanon 
:alarm_clock:：:keycap_2::keycap_0:：:keycap_3::keycap_0::China:
:clapper_board:：IG @MajorGroup 

#MAJORXเตนนน #GMMTV 
#tawan_v #nanon_korapat 
#taytawanfc https://t.co/mXL6iZAKCl
59022 @hachamechaYANO 実は私の父がどうしてもイニシャルをKで続けてほしいとお願いされてまして。。。
なんkで、娘は妻のイニシャル L をとってLisaに。^^
59050 Seungkwan :face_with_tears_of_joy::face_with_tears_of_joy::face_with_tears_of_joy:
#일요일에도_고잉세븐틴 
#GoingSVT_Marathon 
#SEVENTEEN
#GoingSEVENTEEN
@pledis_17
59157 ทุกอย่างสำหรับการจ้างงาน
一切都是为了8000万聘礼！
:UP!_button: https://t.co/FqR7Prbpej
59376 @miharajunco @tamiki2016 在外国 日本人へも10万円の給付をお願いいたします。
59381 ikay nangurog? :face_with_tears_of_joy::face_with_tears_of_joy::face_with_tears_of_joy:
#일요일에도_고잉세븐틴 
#GoingSVT_Marathon 
#SEVENTEEN
#GoingSEVENTEEN
@pledis_17
59511 yesss
#일요일에도_고잉세븐틴 
#GoingSVT_Marathon 
#SEVENTEEN
#GoingSEVENTEEN
@pledis_17 https://t.co/Pq7YH

68580 엄마: 요리를 어떡하지?
작은 동생아: 카보나라
나: lachimolala ? 

:rolling_on_the_floor_laughing::rolling_on_the_floor_laughing: they think I'm crazy because of what I said lol
68775 MJ :smiling_face::face_blowing_a_kiss: #아스트로 @offclASTRO #GATEWAY #ASTRO https://t.co/RTDgwVnHLP
68806 เพียงแค่นั่งและจ้องมอง รออะไรเลย:sunrise_over_mountains:
#staysafe:face_with_medical_mask: 
#stayhome:house: 
#helptoflattenthecurve 
#fightphilippines:Philippines::flexed_biceps_medium-light_skin_tone: @ Basud, Camarines Norte https://t.co/z02HT8Lcft
69254 안녕! :blue_heart: @ Quezon City, Philippines https://t.co/BT5fUgzMtk
69255 あはははは

コンドミニアムの隣人のドア前から大量のビール瓶が消えたと思ったら、今度はコーラとスプライトが山積みになってます！
ダバオはお酒買えないのでしょうがない…
69350 @moriken2311 こんにちは。
マニラ在住の方も変な感じと言っていましたが、人間が活動していんしから何か変化があるんですかね。^^
69415 Before ECQ, I always loved watching my friend cook. So here’s the solution haha. I miss Jen’s 목소리 too. 복515 https://t.co/AyEwNnF5th
69804 小吉はまだ吉です https://t.co/DHt2zB4T9L https://t.co/Dqd2NWZY7E
69863 Dear chaobaan:woman_farmer_li

KeyboardInterrupt: 